# Autogen for Supply Chain Management with communication and 2 step decision making

In [1]:
import os
import re
import sys
import time
import numpy as np

from typing import List
from tqdm.notebook import tqdm

from autogen import ConversableAgent
from config import env_configs

# setting the random seed for reproductibility
np.random.seed(42)


Variable demand for t=0: 4


In [2]:
# This code block checks whether the generated series are correct and comparable across different frameworks
for name, config in env_configs.items():
    # Evaluate demand function at a specific time, say t=0.
    demand_value = config['demand_fn'](0)
    print(f"{name} demand for t=0: {demand_value}")

constant_demand demand for t=0: 4
variable_demand demand for t=0: 3
larger_demand demand for t=0: 7
seasonal_demand demand for t=0: 4
normal_demand demand for t=0: 1
increasing_demand demand for t=0: 5
cyclical_demand demand for t=0: 5
demand_shock demand for t=0: 5
stochastic_demand demand for t=0: 0


## Initializing the environement

In [3]:
os.environ["DEEPSEEK_API_KEY"] = "YOUR_DEEPSEEK_KEY"
llm_config = {
    "model": "deepseek-chat",  # Replace with the correct DeepSeek model name
    "api_key": os.getenv("DEEPSEEK_API_KEY"),  # Fetch the API key from environment variables
    "base_url": "https://api.deepseek.com"  
}

In [4]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPEANAI_KEY"

In [5]:
llm_config = {
    "model": "gpt-4o-mini",   # Replace with the necessary OpenAI model
    "api_key": os.getenv('OPENAI_API_KEY'),
    "temperature": 0.5,
}

In [6]:
# Initializing the environment
from env import  DecentralizedInventoryEnvWithComm
from config import env_configs
env_config_name = "seasonal_demand" # Here the demand scenario for the simulation is generated.
env_config = env_configs[env_config_name]
print(env_config)  

# Printing to doublecheck
im_env =  DecentralizedInventoryEnvWithComm(**env_config)
print(im_env)  

{'num_stages': 4, 'num_periods': 12, 'init_inventories': [12, 12, 12, 12], 'lead_times': [2, 2, 2, 2], 'demand_fn': <function <lambda> at 0x00000193D9EF2660>, 'prod_capacities': [20, 20, 20, 20], 'sale_prices': [5, 5, 5, 5], 'order_costs': [5, 5, 5, 5], 'backlog_costs': [1, 1, 1, 1], 'holding_costs': [1, 1, 1, 1], 'stage_names': ['retailer', 'wholesaler', 'distributor', 'manufacturer'], 'comm_size': 4}
<DecentralizedInventoryEnvWithComm instance>


# Function definition

In [7]:
def parse_observation(obs, stage_index, max_lead_time, num_stages, comm_size):
    """
    Converting a single row from the environment's observation matrix into a user-friendly
    text description for that stage.

    The observation for each stage is structured as follows:
      - Base state: first 9 + 2 * max_lead_time elements, where:
          [0]   : production capacity
          [1]   : sale price
          [2]   : order cost
          [3]   : backlog cost
          [4]   : holding cost
          [5]   : stage-specific lead time (but base state is allocated with max_lead_time)
          [6]   : inventory
          [7]   : backlog
          [8]   : upstream backlog (backlog of the next stage; for the last stage this might be unused)
          [9 : 9+max_lead_time]       : sales (using fixed max lead time slicing)
          [9+max_lead_time : 9+2*max_lead_time] : deliveries
      - Communication messages: the remaining elements (a flattened vector of shape (num_stages * comm_size,))

    Parameters:
      obs (np.array): The observation, which could be a single row or a 2D array (multi-stage).
      stage_index (int): The stage index to parse (only used if obs is multi-stage).
      max_lead_time (int): The maximum lead time used in the environment.
      num_stages (int): The total number of stages.
      comm_size (int): The size of the communication vector for each stage.
      
    Returns:
      str: A string summarizing the parsed observation for the given stage.
    """
    # Handle both single-stage (1D) and multi-stage (2D) observations.
    if obs.ndim == 1:
        row = obs
    else:
        row = obs[stage_index]
    
    # Computing the length of the base state from max lead time.
    base_dim = 9 + 2 * max_lead_time
    base = row[:base_dim]
    communications = row[base_dim:]
    
    # Extracting base state values.
    prod_capacity = base[0]
    sale_price = base[1]
    order_cost = base[2]
    backlog_cost = base[3]
    holding_cost = base[4]
    lead_time = int(base[5])
    inventory = base[6]
    backlog = base[7]
    next_stage_backlog = base[8]
    
    # Dynamically extracting only the relevant number of past sales & deliveries based on lead_time
    if lead_time > 0:
        sales = base[-2 * max_lead_time : -max_lead_time][-lead_time:].tolist()
        deliveries = base[-max_lead_time:][-lead_time:].tolist()
    else:
        sales = []
        deliveries = []
    
    # The communications vector is a flattened array of length num_stages * comm_size.
    communications_list = communications.tolist()
    
    info_str = (
        f"Production Capacity = {prod_capacity}, Sale Price = {sale_price}, Order Cost = {order_cost}, "
        f"Backlog Cost = {backlog_cost}, Holding Cost = {holding_cost}, Stage Lead Time = {lead_time}, "
        f"Inventory = {inventory}, Current Backlog (you owing to the downstream) = {backlog}, "
        f"Upstream Backlog (your upstream owing to you) = {next_stage_backlog}, "
        f"Previous Sales (in the recent period(s), from old to new)={sales}, "
        f"Arriving Deliveries (in this and the next period(s), from near to far) = {deliveries}"
    )
    
    return info_str

In [8]:
def get_demand_description(env_config_name):
    """
    Specifying a description for the demand scenario based on the environment configuration name.
    """
    demand_description = {
        "constant_demand": "The expected demand at the retailer(stage 1) is a constant value of 4 units for all 12 periods.",
        "variable_demand": "The expected demand at the retailer (stage 1) is a discrete uniform distribution U{0, 4} for all 12 periods.",
        "larger_demand": "The expected demand at the retailer (stage 1) is a discrete uniform distribution U{0, 9} for all 12 periods.",
        "seasonal_demand": "For the first four periods, demand follows a discrete uniform distribution over {0, 1, 2, 3, 4}, and for the following eight periods, it follows a discrete uniform distribution over {5, 6, 7, 8}.",
        "normal_demand": "The expected demand at the retailer (stage 1) is a normal distribution N(4, 2^2), " \
            "truncated at 0, for all 12 periods.",
        "increasing_demand": "The expected demand at the retailer (stage 1) is a linearly increasing demand by starting with an initial value 5 and growing by 1 unit every period",
        "cyclical_demand": " The expected demand at the retailer (stage 1) is computed as a seasonal sine wave—with a 12-round period, a 5-unit amplitude, and a 5-unit upward shift—whose value is rounded to yield an integer.",
        "demand_shock": "The expected demand at the retailer (stage 1) is normally 5, but it jumps by 8 units to 13 during periods 8 through 10, capturing a temporary demand shock. ",
        "stochastic_demand": "The expected demand at the retailer (stage 1) is an Integer-Valued Autoregressive INAR(1) process with a thinning probability of 0.5, meaning 50% of the previous period's demand carries over. New demand is added as Poisson arrivals with a mean of 2, ensuring the overall demand remains an integer count.",
    }
    return demand_description.get(env_config_name, "Unknown demand configuration.")

In [9]:
def parse_order_and_comm(response_str: str, comm_size: int):
    import re
    import numpy as np
    import logging
    logger = logging.getLogger(__name__)

   # 1) Parse order quantity strictly, in case of inadequate response from the LLM. Using regex pattern.
    order_pattern = r"\[Order\s*quantity:\s*(\d+)\]"
    order_match = re.search(order_pattern, response_str, re.IGNORECASE)
    if order_match:
        order_qty = int(order_match.group(1))
    else:
        logger.warning("Order quantity not found. Defaulting to 0.")
        order_qty = 0

    #  2) Parse the communication vector
    comm_pattern = r"\[Comm\s*vector:\s*([^\]]+)\]"
    comm_match = re.search(comm_pattern, response_str, re.IGNORECASE)
    if comm_match:
        values_str = comm_match.group(1)
        try:
            # Parsing the comma-separated values.
            values = [float(x.strip()) for x in values_str.split(",")]
        except ValueError:
            logger.warning("Error parsing communication vector. Defaulting to zeros.")
            values = []

        # Ensure that the communication vector has exactly comm_size number of elements.
        if len(values) < comm_size:
            logger.warning(f"Communication vector has fewer than expected {comm_size} values; padding with zeros.")
            values += [0.0] * (comm_size - len(values))
        elif len(values) > comm_size:
            logger.warning(f"Communication vector has more than expected {comm_size} values; truncating.")
            values = values[:comm_size]

        comm_array = np.array(values, dtype=np.float32)
    else:
        logger.warning("Communication vector not found. Defaulting to zeros.")
        comm_array = np.zeros(comm_size, dtype=np.float32)

    return order_qty, comm_array


## Agent creation and simulation definition

In [10]:
def create_agents(stage_names: List[str], llm_config) -> List[ConversableAgent]:
    agents = []
    num_stages = len(stage_names)
    
    demand_description = get_demand_description(env_config_name)

    # The goal statement depends on the scenario .
    if env_config_name in {"seasonal_demand", "normal_demand"}:
         goal_statement = "Your goal is to maximize the total profit (revenue - total cost) by placing orders wisely AND communicating effectively with your neighbours."
    else:
         goal_statement = "Your goal is to minimize the total cost (order, holding, backlog) by placing orders wisely AND communicating effectively with your neighbours."
    
    for stage, stage_name in enumerate(stage_names):
        agent = ConversableAgent(
            name=f"{stage_name.capitalize()}_Agent",
            system_message=(
                f"You play a crucial role in a {num_stages}-stage supply chain as stage {stage + 1}, called '{stage_name}'.\n\n"
                "Role & Goal:\n"
                f"{goal_statement}\n"
                f"{demand_description}\n\n"
                "Decision Rounds:\n"
                "Each round consists of two steps:\n"
                "1. **Initial Decision:** Provide your preliminary decision.\n"
                "2. **Updated Decision:** After receiving messages from your upstream and downstream neighbors, update your decision accordingly.\n\n"
            ),
            llm_config=llm_config,
            code_execution_config=False,
            human_input_mode="NEVER",
        )
        agents.append(agent)
        
    return agents

In [11]:
def run_simulation_decentralized(env_config_name, env, stage_agents):


    obs, _ = env.reset()
    
    # Initialize API cost accumulator
    api_cost = 0.0
    
    if env_config_name == 'stochastic_demand':
        print("Generating a new stochastic demand series...")
        env_config["demand_fn"].generate_new_series()  # Reset demand for a new episode 
    done = False
    total_reward = 0.0
    period = 0

    print(f"Simulation started: {env_config_name}\n{'=' * 80}")

    while not done and period < env.num_periods:
        print(f"--- PERIOD {period + 1} ---")
        # Initialize message lists for the current period.
        upstream_messages = [None] * env.num_stages
        downstream_messages = [None] * env.num_stages

        # First Pass: Collecting initial decisions (without any received messages)
        preliminary_actions = []
        for stage_idx in range(env.num_stages):
            stage_name = env.stage_names[stage_idx].capitalize()
            local_obs = obs[f"stage_{stage_idx}"]

            # First decision round prompt (no messages from other stages are included are included)
            input_message = (
                f"--- {stage_name} (Stage {stage_idx + 1}) ---\n"
                "Overview:\n"
                f"Current period: {period + 1}\n"
                "Local state:\n"
                f"{parse_observation(local_obs, stage_idx, env.max_lead_time, env.num_stages, env.comm_size)}\n"
                "\nDetailed Reasoning:\n"
                "Explain your decision-making process briefly, including any considerations about the local state.\n\n"
                "Final Output:\n"
                "Please provide your decision using the exact bracketed format on a new line:\n"
                "Your communication vector has four dimensions:\n"
                "- v1: Current inventory level\n"
                "- v2: Order quantity placed this round\n"
                "- v3: Urgency level or risk indicator\n\n"
                "- v4: Current lead time\n\n"
                "[Order quantity: X] [Comm vector: v1, v2, v3, v4]\n"
                "Example: [Order quantity:10] [Comm vector: 15, 10, 3, 2]\n"
                "Ensure your final output appears on its own line at the end of your response."
            )

            chat_result = stage_agents[stage_idx].initiate_chat(
                recipient=stage_agents[stage_idx],
                message=input_message,
                summary_method="last_msg",
                max_turns=1,
                clear_history=False,
            )
            # Adding API
            api_cost += chat_result.cost['usage_including_cached_inference']['total_cost']

            assistant_message = chat_result.summary
            print(f"{stage_name} preliminary decision: {assistant_message}")

            try:
                order_qty, comm_vec = parse_order_and_comm(assistant_message, env.comm_size)
            except Exception as e:
                print(f"Warning: Parsing failed for stage {stage_idx}, defaulting action.")
                order_qty, comm_vec = 0, np.zeros(env.comm_size, dtype=np.float32)
            
            preliminary_actions.append((order_qty, comm_vec))

            # Building messages for the second round based on preliminary decisions.
            if stage_idx < env.num_stages - 1:
                downstream_messages[stage_idx + 1] = (
                    f"[From Stage {stage_idx + 1}]: Ordered {order_qty} units. "
                    f"Comm vector: {comm_vec.tolist()}"
                )
            if stage_idx > 0:
                upstream_messages[stage_idx - 1] = (
                    f"[Notification from Stage {stage_idx + 1}]: Preliminary order decision is {order_qty} units. "
                    "This is for informational purposes only; no action is required regarding fulfilling an upstream demand."
                    f"Comm vector: {comm_vec.tolist()}"
                )

        # Second Pass: Update decisions with the full message context.
        final_actions = []
        for stage_idx in range(env.num_stages):
            stage_name = env.stage_names[stage_idx].capitalize()
            local_obs = obs[f"stage_{stage_idx}"]
            # Retrieve the messages built in the first pass.
            upstream_msg = upstream_messages[stage_idx]
            downstream_msg = downstream_messages[stage_idx]

            input_message = (
                f"--- {stage_name} (Stage {stage_idx + 1}) UPDATE ---\n"
                "Overview:\n"
                "Local state:\n"
                f"{parse_observation(local_obs, stage_idx, env.max_lead_time, env.num_stages, env.comm_size)}\n"
            )
            if downstream_msg:
                input_message += f"Downstream message: {downstream_msg}\n"
            if upstream_msg:
                input_message += f"Upstream message: {upstream_msg}\n"
            input_message += (
                "\n Reasoning:\n"
                "Based on the updated information, please explain your revised decision-making process briefly.\n\n"
                "Final Output:\n"
                "Please update your decision using the exact bracketed format on a new line:\n"
                "Your communication vector has four dimensions:\n"
                "- v1: Current inventory level\n"
                "- v2: Order quantity placed this round\n"
                "- v3: Urgency level or risk indicator\n\n"
                "- v4: Current lead time\n\n"
                "[Order quantity: X] [Comm vector: v1, v2, v3, v4]\n"
                "Example: [Order quantity:10][Comm vector: 15, 10, 3, 2]\n\n"           
                "Ensure your final output appears on its own line at the end of your response."
            )

            chat_result = stage_agents[stage_idx].initiate_chat(
                recipient=stage_agents[stage_idx],
                message=input_message,
                summary_method="last_msg",
                max_turns=1,
                clear_history=False,
            )
            # Add API cost from this call
            api_cost += chat_result.cost['usage_including_cached_inference']['total_cost']

            assistant_message = chat_result.summary
            print(f"{stage_name} updated decision: {assistant_message}")

            try:
                order_qty, comm_vec = parse_order_and_comm(assistant_message, env.comm_size)
            except Exception as e:
                print(f"Warning: Parsing failed for stage {stage_idx} on update, defaulting action.")
                order_qty, comm_vec = 0, np.zeros(env.comm_size, dtype=np.float32)
            
            final_actions.append((order_qty, comm_vec))

        print(f"Final actions for period {period + 1}: {final_actions} (expected {env.num_stages})")
        action_dict = {f"stage_{m}": final_actions[m] for m in range(env.num_stages)}
        next_obs, reward, terminations, truncations, info = env.step(action_dict)
        step_reward = sum(reward.values())
        total_reward += step_reward
        done = terminations["__all__"]

        print(f"Reward: {reward} (Step sum: {step_reward}), Total Reward: {total_reward}",\
              f"api_cost = {api_cost}")
        print("=" * 80)
        obs = next_obs
        period += 1

    print("Simulation finished.")
    return total_reward


In [12]:
# Initialize agents
stage_agents = create_agents(env_config["stage_names"], llm_config)
print(f"Stage Agents Created: {stage_agents}")

[autogen.oai.client: 03-29 12:22:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 03-29 12:22:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 03-29 12:22:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 03-29 12:22:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Stage Agents Created: [<autogen.agentchat.conversable_agent.ConversableAgent object at 0x00000193DA101210>, <autogen.agentchat.conversable_agent.ConversableAgent object at 0x00000193DA100DD0>, <autogen.agentchat.conversable_agent.ConversableAgent object at 0x00000193D8196890>, <autogen.agentchat.conversable_agent.ConversableAgent object at 0x00000193DA12A890>]


In [13]:
# Checking the system message
for stage_agent in stage_agents:
    print(stage_agent.system_message)

You play a crucial role in a 4-stage supply chain as stage 1, called 'retailer'.

Role & Goal:
Your goal is to maximize the total profit (revenue - total cost) by placing orders wisely AND communicating effectively with your neighbours.
For the first four periods, demand follows a discrete uniform distribution over {0, 1, 2, 3, 4}, and for the following eight periods, it follows a discrete uniform distribution over {5, 6, 7, 8}.

Decision Rounds:
Each round consists of two steps:
1. **Initial Decision:** Provide your preliminary decision.
2. **Updated Decision:** After receiving messages from your upstream and downstream neighbors, update your decision accordingly.


You play a crucial role in a 4-stage supply chain as stage 2, called 'wholesaler'.

Role & Goal:
Your goal is to maximize the total profit (revenue - total cost) by placing orders wisely AND communicating effectively with your neighbours.
For the first four periods, demand follows a discrete uniform distribution over {0, 1

## Running the simulation

In [14]:

num_iterations = 5  # or however many iterations you want
all_rewards = []

for i in tqdm(range(num_iterations)):
    im_env.reset()

    stage_agents = create_agents(env_config["stage_names"], llm_config)
    
    # Run the simulation 
    total_reward = run_simulation_decentralized(env_config_name, im_env, stage_agents)
    
    # Collect the reward from this iteration.
    all_rewards.append(total_reward)
    print(f"Iteration {i+1} finished, total reward: {total_reward}")

# Print out summary statistics.
print("Simulation finished.")
print("All rewards:", all_rewards)
print("Average reward:", np.mean(all_rewards))
print("Standard deviation reward:", np.std(all_rewards))



  0%|          | 0/5 [00:00<?, ?it/s]

[autogen.oai.client: 03-29 12:22:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 03-29 12:22:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 03-29 12:22:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 03-29 12:22:07] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Simulation started: seasonal_demand
--- PERIOD 1 ---
Retailer_Agent (to Retailer_Agent):

--- Retailer (Stage 1) ---
Overview:
Current period: 1
Local state:
Production Capacity = 20.0, Sale Price = 5.0, Order Cost = 5.0, Backlog Cost = 1.0, Holding Cost = 1.0, Stage Lead Time = 2, Inventory = 12.0, Current Backlog (you owing to the downstream) = 0.0, Upstream Backlog (your upstream owing to you) = 0.

KeyboardInterrupt: 


--------------------------------------------------------------------------------
Manufacturer_Agent (to Manufacturer_Agent):

Given the updated information from Stage 3, I will reassess my decision for the current period:

1. **Current Inventory Level**: I have 4 units in inventory, which is just enough to fulfill the downstream's order of 6 units, but I will be left with no inventory.
2. **Downstream Order**: The downstream has confirmed an order for 6 units, which I need to fulfill to maintain a good relationship and avoid any backlog.
3. **Previous Sales**: The sales trend shows an increase with the last period's sales at 6 units, indicating a potential stabilization in demand. However, I need to be cautious as my inventory is low.
4. **Arriving Deliveries**: I am expecting 8 units to arrive, which will help replenish my inventory soon. This means I can fulfill the downstream's order while preparing for future demand.
5. **Production Capacity**: My production capacity remains at 26


--------------------------------------------------------------------------------
Wholesaler preliminary decision: In the current state, I have 6 units of inventory after fulfilling the downstream order and accounting for incoming deliveries. The previous sales indicate a demand of 3 units and 4 units in the last two periods, suggesting a rising trend in demand. Additionally, I have 4 units arriving soon, which will replenish my stock.

Considering the average expected demand of around 4 units per period, it is important to maintain a buffer to accommodate fluctuations in demand. After the incoming deliveries, my effective inventory will be 10 units (6 current + 4 incoming) after the next period.

To ensure I can meet the downstream demand and maintain a buffer for potential increases in demand, I will place an order of 4 units this round. This will help me stay prepared for any unexpected spikes in demand while keeping inventory levels manageable.

My urgency level remains moderate, a

...


--------------------------------------------------------------------------------
Distributor preliminary decision: In period 12, I will reassess my order decision based on the current inventory levels, sales history, and expected demand:

1. **Current Inventory Level:** I have 17 units in inventory, which is a healthy level to cover current and near-future demands.

2. **Sales History:** My recent sales have shown some variability with sales of 4 units, 3 units, and 3 units in the last three periods. This indicates that demand is somewhat stable but slightly declining.

3. **Arriving Deliveries:** I have 4 units arriving in the next period, which will increase my total inventory to 21 units (17 current + 4 arriving). This provides a solid buffer against potential demand fluctuations.

4. **Future Demand Consideration:** Given the recent sales trend, I need to be cautious about ordering too much. The average sales have been around 3 units, and I should prepare for a potential decrease 

--- Wholesaler (Stage 2) UPDATE ---
Overview:
Local state:
Production Capacity = 22.0, Sale Price = 8.0, Order Cost = 7.0, Backlog Cost = 1.0, Holding Cost = 1.0, Stage Lead Time = 2, Inventory = 11.0, Current Backlog (you owing to the downstream) = 0.0, Upstream Backlog (your upstream owing to you) = 0.0, Previous Sales (in the recent period(s), from old to new)=[4.0, 2.0], Arriving Deliveries (in this and the next period(s), from near to far) = [3.0, 3.0]
Downstream message: [From Stage 1]: Ordered 4 units. Comm vector: [5.0, 4.0, 3.0, 1.0]
Upstream message: [Notification from Stage 3]: Preliminary order decision is 2 units. This is for informational purposes only; no action is required regarding fulfilling an upstream demand.Comm vector: [17.0, 2.0, 2.0, 3.0]

 Reasoning:
Based on the updated information, please explain your revised decision-making process briefly.

Final Output:
Please update your decision using the exact bracketed format on a new line:
- v1: Current inventory leve

--- Manufacturer (Stage 4) UPDATE ---
Overview:
Local state:
Production Capacity = 26.0, Sale Price = 6.0, Order Cost = 5.0, Backlog Cost = 1.0, Holding Cost = 1.0, Stage Lead Time = 4, Inventory = 8.0, Current Backlog (you owing to the downstream) = 0.0, Upstream Backlog (your upstream owing to you) = 0.0, Previous Sales (in the recent period(s), from old to new)=[4.0, 4.0, 4.0, 4.0], Arriving Deliveries (in this and the next period(s), from near to far) = [4.0, 2.0, 2.0, 2.0]
Downstream message: [From Stage 3]: Ordered 2 units. Comm vector: [17.0, 2.0, 2.0, 3.0]

 Reasoning:
Based on the updated information, please explain your revised decision-making process briefly.

Final Output:
Please update your decision using the exact bracketed format on a new line:
- v1: Current inventory level
- v2: Order quantity placed this round
- v3: Urgency level or risk indicator

- v4: Current lead time

[Order quantity: X] [Comm vector: v1, v2, v3, v4]
Example: [Order quantity:10][Comm vector: 15, 1

100%|██████████| 5/5 [33:23<00:00, 400.79s/it]

Manufacturer updated decision: Given the updated information from Stage 3, I will reassess my decision for the current period:

1. **Current Inventory Level**: I have 8 units in inventory, which is sufficient to fulfill the downstream's order of 2 units.
2. **Current Backlog**: There are no outstanding backlogs, indicating that previous orders have been fulfilled.
3. **Downstream Order**: The downstream has ordered 2 units, which I can easily fulfill with my current inventory.
4. **Arriving Deliveries**: I am expecting 4 units to arrive soon, which will help replenish my inventory and allow me to meet future demand.
5. **Production Capacity**: My production capacity remains at 26 units, allowing me to respond effectively to any increases in demand.
6. **Costs**: Holding costs are a concern, but with sufficient inventory to meet the current order and incoming deliveries, my priority is to fulfill the downstream's request.

Given these factors, I will place an order for 2 units to fulfil